<a href="https://colab.research.google.com/github/Mc4minta/AIB5-PcapAttackClassifier/blob/main/ProcessTestSetPCAP_FTPSSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ftpbrute.zip, sshbrute.zip >> .csv

# ⏲ CICFLOWMETER v.3

In [2]:
!sudo apt-get install libpcap-dev # install libpcap-dev
!wget https://codeberg.org/iortega/TCPDUMP_and_CICFlowMeter/archive/master:CICFlowMeters/CICFlowMeter-3.0.zip # download CICFlowMeter3.0.zip
!unzip CICFlowMeter-3.0.zip -d CICFlowMeter-3.0 # Extract CICFlowMeter3.0.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libdbus-1-dev libpcap0.8 libpcap0.8-dev
The following NEW packages will be installed:
  libdbus-1-dev libpcap-dev libpcap0.8 libpcap0.8-dev
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 607 kB of archives.
After this operation, 2,238 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8 amd64 1.10.1-4ubuntu1.22.04.1 [145 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libdbus-1-dev amd64 1.12.20-2ubuntu4.1 [188 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap0.8-dev amd64 1.10.1-4ubuntu1.22.04.1 [270 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpcap-dev amd64 1.10.1-4ubuntu1.22.04.1 [3,326 B]
Fetched 607 kB in 0s (4,320 kB/s)
debconf: unable to initialize frontend: Dialog
d

In [3]:
import os

# Define the directory paths
in_directory = 'data/in'
out_directory = 'data/out'

# Create the directories if they don't exist
os.makedirs(in_directory, exist_ok=True)
os.makedirs(out_directory, exist_ok=True)

print(f"Directories '{in_directory}' and '{out_directory}' created (if they didn't exist).")

Directories 'data/in' and 'data/out' created (if they didn't exist).


# ✅ FTPBruteforce - GET PCAP FILES

In [6]:
import os
directory_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline'

# Check if the directory exists before listing its contents
if os.path.exists(directory_path):
  # List the contents of the directory
  contents = os.listdir(directory_path)

  # Print the contents
  for item in contents:
    print(item)
else:
  print(f"Directory not found: {directory_path}")

DoS.zip
DDoS.zip
PortScan.zip
Benign.zip
FTPBruteforce.zip
SSHBruteforce.zip
DoS-HTTP-Flood.zip
DoS-SlowRate.zip
Benign2.zip
DoS-overall.zip
DoS-Layer3and4.zip


In [7]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline/FTPBruteforce.zip'
destination_directory = '/content/'

# Unzip the file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_directory)
    print(f"'{zip_file_path}' unzipped to '{destination_directory}'.")
except FileNotFoundError:
    print(f"Error: Zip file not found at '{zip_file_path}'")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
except Exception as e:
    print(f"An error occurred: {e}")

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline/FTPBruteforce.zip' unzipped to '/content/'.


# ✅ FTPBruteforce - Convert to .csv

In [8]:
import os

dos_directory = 'FTPBruteforce'
pcap_files = [f for f in os.listdir(dos_directory) if f.endswith('.pcap')]

if pcap_files:
    print(f"Found {len(pcap_files)} .pcap files in '{dos_directory}':")
    for pcap_file in pcap_files:
        print(pcap_file)
else:
    print(f"No .pcap files found in '{dos_directory}'.")

Found 3 .pcap files in 'FTPBruteforce':
ftpbrute-ubuntu.pcap
ftpbrute-kali.pcap
bruteforce_ftp.pcap


In [9]:
import os
import shutil
import subprocess

# Define the attack type directory (change this variable for different attack types)
attack_type = 'FTPBruteforce' # You can change this to 'DDoS', 'PortScan', etc.

attack_directory = attack_type
data_in_directory = 'data/in'
data_out_directory = 'data/out'
csv_destination_directory = f'{attack_type}_csv'  # Directory for CSV files based on attack type
cicflowmeter_command = './CICFlowMeter-3.0/tcpdump_and_cicflowmeter/bin/CICFlowMeter'

# Create the CSV destination directory if it doesn't exist
os.makedirs(csv_destination_directory, exist_ok=True)
print(f"Directory '{csv_destination_directory}' created (if it didn't exist).")

# Ensure the attack directory exists
if not os.path.exists(attack_directory):
    print(f"Error: Attack directory '{attack_directory}' not found.")
else:
    # Get the list of pcap files in the attack directory
    pcap_files = [f for f in os.listdir(attack_directory) if f.endswith('.pcap')]

    if pcap_files:
        print(f"Processing {len(pcap_files)} .pcap files in '{attack_directory}'...")
        for pcap_file in pcap_files:
            source_path = os.path.join(attack_directory, pcap_file)
            dest_in_path = os.path.join(data_in_directory, pcap_file)

            print(f"\nProcessing: {pcap_file}")

            # Step 2a: Move to data/in
            print(f"Moving '{pcap_file}' to '{data_in_directory}'...")
            shutil.move(source_path, data_in_directory)

            # Step 2b: Run CICFlowMeter
            print("Running CICFlowMeter...")
            try:
                # The CICFlowMeter command expects the input directory to be 'data/in' and outputs to 'data/out'
                subprocess.run([cicflowmeter_command], check=True, capture_output=True, text=True)
                print("CICFlowMeter finished successfully.")
            except subprocess.CalledProcessError as e:
                print(f"Error running CICFlowMeter: {e}")
                print(f"Stdout: {e.stdout}")
                print(f"Stderr: {e.stderr}")
                # Move the file back even if there was an error
                print(f"Moving '{pcap_file}' back to '{attack_directory}' due to error.")
                shutil.move(dest_in_path, attack_directory)
                continue # Move to the next file

            # Step 2c: Move results - Move CSV to DoS_csv and original pcap back to DoS
            print(f"Moving results from '{data_out_directory}' to '{csv_destination_directory}' and original pcap back to '{attack_directory}'...")
            output_files = os.listdir(data_out_directory)
            for output_file in output_files:
                # Check if the file is a CSV file generated by CICFlowMeter
                if output_file.endswith('_ISCX.csv'):
                    source_out_path = os.path.join(data_out_directory, output_file)
                    dest_csv_path = os.path.join(csv_destination_directory, output_file)
                    shutil.move(source_out_path, dest_csv_path)
                    print(f"Moved '{output_file}' to '{csv_destination_directory}'.")

            # Move the original pcap file back to the attack directory
            # We need to find the file in data/in after processing.
            # Assuming only one pcap file is in data/in at a time during the loop.
            processed_pcap_files_in_in = os.listdir(data_in_directory)
            for processed_pcap_file in processed_pcap_files_in_in:
                 if processed_pcap_file.endswith('.pcap'):
                    processed_in_path = os.path.join(data_in_directory, processed_pcap_file)
                    original_source_path = os.path.join(attack_directory, processed_pcap_file) # Construct original path
                    shutil.move(processed_in_path, original_source_path) # Move from data/in back to original location
                    print(f"Moved '{processed_pcap_file}' back to '{attack_directory}'.")


            # Step 2d: Clean up data/in and data/out
            print("Cleaning up temporary directories...")
            # Clean up any remaining files in data/out (should have been moved, but just in case)
            remaining_out_files = os.listdir(data_out_directory)
            for file in remaining_out_files:
                 os.remove(os.path.join(data_out_directory, file))

            print("Cleanup complete.")


        print(f"\nFinished processing all .pcap files in the '{attack_directory}' folder.")

    else:
        print(f"No .pcap files found in '{attack_directory}' to process.")

Directory 'FTPBruteforce_csv' created (if it didn't exist).
Processing 3 .pcap files in 'FTPBruteforce'...

Processing: ftpbrute-ubuntu.pcap
Moving 'ftpbrute-ubuntu.pcap' to 'data/in'...
Running CICFlowMeter...
CICFlowMeter finished successfully.
Moving results from 'data/out' to 'FTPBruteforce_csv' and original pcap back to 'FTPBruteforce'...
Moved 'ftpbrute-ubuntu_ISCX.csv' to 'FTPBruteforce_csv'.
Moved 'ftpbrute-ubuntu.pcap' back to 'FTPBruteforce'.
Cleaning up temporary directories...
Cleanup complete.

Processing: ftpbrute-kali.pcap
Moving 'ftpbrute-kali.pcap' to 'data/in'...
Running CICFlowMeter...
CICFlowMeter finished successfully.
Moving results from 'data/out' to 'FTPBruteforce_csv' and original pcap back to 'FTPBruteforce'...
Moved 'ftpbrute-kali_ISCX.csv' to 'FTPBruteforce_csv'.
Moved 'ftpbrute-kali.pcap' back to 'FTPBruteforce'.
Cleaning up temporary directories...
Cleanup complete.

Processing: bruteforce_ftp.pcap
Moving 'bruteforce_ftp.pcap' to 'data/in'...
Running CICFl

In [10]:
! zip -r FTPBruteforce_csv.zip FTPBruteforce_csv

  adding: FTPBruteforce_csv/ (stored 0%)
  adding: FTPBruteforce_csv/ftpbrute-ubuntu_ISCX.csv (deflated 82%)
  adding: FTPBruteforce_csv/ftpbrute-kali_ISCX.csv (deflated 83%)
  adding: FTPBruteforce_csv/bruteforce_ftp_ISCX.csv (deflated 82%)


# ✅ SSHBruteforce - GET PCAP FILES

In [12]:
import os
directory_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline'

# Check if the directory exists before listing its contents
if os.path.exists(directory_path):
  # List the contents of the directory
  contents = os.listdir(directory_path)

  # Print the contents
  for item in contents:
    print(item)
else:
  print(f"Directory not found: {directory_path}")

DoS.zip
DDoS.zip
PortScan.zip
Benign.zip
FTPBruteforce.zip
SSHBruteforce.zip
DoS-HTTP-Flood.zip
DoS-SlowRate.zip
Benign2.zip
DoS-overall.zip
DoS-Layer3and4.zip


In [13]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline/SSHBruteforce.zip'
destination_directory = '/content/'

# Unzip the file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_directory)
    print(f"'{zip_file_path}' unzipped to '{destination_directory}'.")
except FileNotFoundError:
    print(f"Error: Zip file not found at '{zip_file_path}'")
except zipfile.BadZipFile:
    print(f"Error: '{zip_file_path}' is not a valid zip file.")
except Exception as e:
    print(f"An error occurred: {e}")

'/content/drive/MyDrive/Share to Mc4/AIBuilders5-MiN/TestsetForBaseline/SSHBruteforce.zip' unzipped to '/content/'.


# ✅ SSHBruteforce - Convert to .csv

In [14]:
import os

dos_directory = 'SSHBruteforce'
pcap_files = [f for f in os.listdir(dos_directory) if f.endswith('.pcap')]

if pcap_files:
    print(f"Found {len(pcap_files)} .pcap files in '{dos_directory}':")
    for pcap_file in pcap_files:
        print(pcap_file)
else:
    print(f"No .pcap files found in '{dos_directory}'.")

Found 3 .pcap files in 'SSHBruteforce':
sshbrute-kali.pcap
bruteforce_ssh.pcap
sshbrute-ubuntu.pcap


In [15]:
import os
import shutil
import subprocess

# Define the attack type directory (change this variable for different attack types)
attack_type = 'SSHBruteforce' # You can change this to 'DDoS', 'PortScan', etc.

attack_directory = attack_type
data_in_directory = 'data/in'
data_out_directory = 'data/out'
csv_destination_directory = f'{attack_type}_csv'  # Directory for CSV files based on attack type
cicflowmeter_command = './CICFlowMeter-3.0/tcpdump_and_cicflowmeter/bin/CICFlowMeter'

# Create the CSV destination directory if it doesn't exist
os.makedirs(csv_destination_directory, exist_ok=True)
print(f"Directory '{csv_destination_directory}' created (if it didn't exist).")

# Ensure the attack directory exists
if not os.path.exists(attack_directory):
    print(f"Error: Attack directory '{attack_directory}' not found.")
else:
    # Get the list of pcap files in the attack directory
    pcap_files = [f for f in os.listdir(attack_directory) if f.endswith('.pcap')]

    if pcap_files:
        print(f"Processing {len(pcap_files)} .pcap files in '{attack_directory}'...")
        for pcap_file in pcap_files:
            source_path = os.path.join(attack_directory, pcap_file)
            dest_in_path = os.path.join(data_in_directory, pcap_file)

            print(f"\nProcessing: {pcap_file}")

            # Step 2a: Move to data/in
            print(f"Moving '{pcap_file}' to '{data_in_directory}'...")
            shutil.move(source_path, data_in_directory)

            # Step 2b: Run CICFlowMeter
            print("Running CICFlowMeter...")
            try:
                # The CICFlowMeter command expects the input directory to be 'data/in' and outputs to 'data/out'
                subprocess.run([cicflowmeter_command], check=True, capture_output=True, text=True)
                print("CICFlowMeter finished successfully.")
            except subprocess.CalledProcessError as e:
                print(f"Error running CICFlowMeter: {e}")
                print(f"Stdout: {e.stdout}")
                print(f"Stderr: {e.stderr}")
                # Move the file back even if there was an error
                print(f"Moving '{pcap_file}' back to '{attack_directory}' due to error.")
                shutil.move(dest_in_path, attack_directory)
                continue # Move to the next file

            # Step 2c: Move results - Move CSV to DoS_csv and original pcap back to DoS
            print(f"Moving results from '{data_out_directory}' to '{csv_destination_directory}' and original pcap back to '{attack_directory}'...")
            output_files = os.listdir(data_out_directory)
            for output_file in output_files:
                # Check if the file is a CSV file generated by CICFlowMeter
                if output_file.endswith('_ISCX.csv'):
                    source_out_path = os.path.join(data_out_directory, output_file)
                    dest_csv_path = os.path.join(csv_destination_directory, output_file)
                    shutil.move(source_out_path, dest_csv_path)
                    print(f"Moved '{output_file}' to '{csv_destination_directory}'.")

            # Move the original pcap file back to the attack directory
            # We need to find the file in data/in after processing.
            # Assuming only one pcap file is in data/in at a time during the loop.
            processed_pcap_files_in_in = os.listdir(data_in_directory)
            for processed_pcap_file in processed_pcap_files_in_in:
                 if processed_pcap_file.endswith('.pcap'):
                    processed_in_path = os.path.join(data_in_directory, processed_pcap_file)
                    original_source_path = os.path.join(attack_directory, processed_pcap_file) # Construct original path
                    shutil.move(processed_in_path, original_source_path) # Move from data/in back to original location
                    print(f"Moved '{processed_pcap_file}' back to '{attack_directory}'.")


            # Step 2d: Clean up data/in and data/out
            print("Cleaning up temporary directories...")
            # Clean up any remaining files in data/out (should have been moved, but just in case)
            remaining_out_files = os.listdir(data_out_directory)
            for file in remaining_out_files:
                 os.remove(os.path.join(data_out_directory, file))

            print("Cleanup complete.")


        print(f"\nFinished processing all .pcap files in the '{attack_directory}' folder.")

    else:
        print(f"No .pcap files found in '{attack_directory}' to process.")

Directory 'SSHBruteforce_csv' created (if it didn't exist).
Processing 3 .pcap files in 'SSHBruteforce'...

Processing: sshbrute-kali.pcap
Moving 'sshbrute-kali.pcap' to 'data/in'...
Running CICFlowMeter...
CICFlowMeter finished successfully.
Moving results from 'data/out' to 'SSHBruteforce_csv' and original pcap back to 'SSHBruteforce'...
Moved 'sshbrute-kali_ISCX.csv' to 'SSHBruteforce_csv'.
Moved 'sshbrute-kali.pcap' back to 'SSHBruteforce'.
Cleaning up temporary directories...
Cleanup complete.

Processing: bruteforce_ssh.pcap
Moving 'bruteforce_ssh.pcap' to 'data/in'...
Running CICFlowMeter...
CICFlowMeter finished successfully.
Moving results from 'data/out' to 'SSHBruteforce_csv' and original pcap back to 'SSHBruteforce'...
Moved 'bruteforce_ssh_ISCX.csv' to 'SSHBruteforce_csv'.
Moved 'bruteforce_ssh.pcap' back to 'SSHBruteforce'.
Cleaning up temporary directories...
Cleanup complete.

Processing: sshbrute-ubuntu.pcap
Moving 'sshbrute-ubuntu.pcap' to 'data/in'...
Running CICFlow

In [16]:
! zip -r SSHBruteforce_csv.zip SSHBruteforce_csv

  adding: SSHBruteforce_csv/ (stored 0%)
  adding: SSHBruteforce_csv/sshbrute-kali_ISCX.csv (deflated 82%)
  adding: SSHBruteforce_csv/sshbrute-ubuntu_ISCX.csv (deflated 81%)
  adding: SSHBruteforce_csv/bruteforce_ssh_ISCX.csv (deflated 80%)
